**目次**<a id='toc0_'></a>    
- 1. [科学技術計算5課題](#toc1_)    
  - 1.1. [課題05-1：レイリー商反復によるQR法による固有値分解の改善](#toc1_1_)    
  - 1.2. [課題05-2：ハウスホルダー変換によるQR分解](#toc1_2_)    
    - 1.2.1. [2つのハウスホルダー変換](#toc1_2_1_)    
      - 1.2.1.1. [「対称行列を三重対角行列に変換する」ためのハウスホルダー法の復習](#toc1_2_1_1_)    
      - 1.2.1.2. [「一般の行列をQR分解する」ためのハウスホルダー変換](#toc1_2_1_2_)    
    - 1.2.2. [課題内容](#toc1_2_2_)    
  - 1.3. [課題05-3：QR分解を用いたthin SVD](#toc1_3_)    
    - 1.3.1. [full QR分解とreduced QR分解](#toc1_3_1_)    
      - 1.3.1.1. [QR分解：$m=n$の場合](#toc1_3_1_1_)    
      - 1.3.1.2. [QR分解：$m>n$の場合](#toc1_3_1_2_)    
      - 1.3.1.3. [QR分解：$m<n$の場合](#toc1_3_1_3_)    
      - 1.3.1.4. [reduced QR分解のまとめ](#toc1_3_1_4_)    
      - 1.3.1.5. [numpy/scipyの実装](#toc1_3_1_5_)    
    - 1.3.2. [full/reduced QR分解を利用したthin SVDの単純な計算方法](#toc1_3_2_)    
    - 1.3.3. [課題内容](#toc1_3_3_)    
  - 1.4. [課題05-4：ゲルシュゴリンの定理による固有値の存在範囲の可視化](#toc1_4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[科学技術計算5課題](#toc0_)

**各課題で共通する注意事項**

- 関数にはアノテーション（引数や返り値の型情報）と`docstring`（関数の説明文）を必ず書く．


## 1.1. <a id='toc1_1_'></a>[課題05-1：レイリー商反復によるQR法による固有値分解の改善](#toc0_)

授業資料で説明したQR法は，まずハウスホルダー変換を使って行列を三重対角行列に変換し，その後，ギブンス変換を使って三重対角行列のQR分解を反復する方法である．これにより，全ての固有値と固有ベクトル（の近似値）が計算される．

それを実装した関数が授業資料の`eigh_by_qr_method()`である．
この関数でのQR法の収束条件として「**主対角成分(`diagonals`)が変化しなくなるまで**」と設定していたが，この条件では収束が不十分であり，非対角成分が完全には0に収束しない．そこで，QR法の収束の精度を上げるために「**優対角成分(`superdiagonals`)も変化しなくなるまで**」という条件を追加しているが，これによりQR法の反復回数が増加してしまう．

そこでこの課題では，以下の方法を考える．

- まず，QR法の反復における収束条件を「**主対角成分(`diagonals`)が変化しなくなるまで**」のみとする．
- 次に，その時点で得られる近似固有値（主対角成分）と近似固有ベクトルを初期値として**レイリー商反復法**を適用し，より精度の高い固有値と固有ベクトルを得る方法を実装する．

つまり，以下のようなアルゴリズムとなる．

1. $n \times n$実対称行列$A$に対して，QR法`eigh_by_qr_method()`により，$n$個の近似固有値`d`と$n$個の近似固有ベクトル`U`を求める．
2. $i = 1, 2, \ldots, n$について
   1. 第$i$近似固有値と第$i$近似固有ベクトルを，レイリー商反復`rayleigh_quotient_iteration()`の初期値に設定する．
   2. レイリー商反復により，更新した第$i$固有値と第$i$固有ベクトルを求める．
3. $n$個の固有値`d`と$n$個の固有ベクトル`U`を返す．


**やること**

1. この方法で`A`の固有値分解を行う処理を，1つの関数`eigh_via_qr_rayleigh()`として実装する．
   - この関数ではまず近似的にQR法を行い，次にレイリー商反復法を使って固有値と固有ベクトルを更新する．
   - 関数の返り値は，固有値のベクトルと，固有ベクトルからなる直交行列とする．これは`np.linalg.eigh()`と同じ形式にする．
2. 実装した`eigh_via_qr_rayleigh()`の結果と，`np.linalg.eigh()`の結果が一致しているかどうか確認する．もし一致しない場合は，その原因を考察する．

**関数定義**

実装する関数`eigh_via_qr_rayleigh()`の入出力は，
`eigh_by_qr_method()`と同じとする．

```python
def eigh_via_qr_rayleigh(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
```

- 実装する関数は引数に行列$A$をとる．
- 実装する関数の返り値は，$A$の固有値分解$U D U^T$の結果である$D$と$U$である．
    - $D$は対角行列ではなく，対角要素のベクトルをndarrayとして返す．


**実装上の注意点**

- この資料のコードが利用できるなら，そのまま用いてよい（むしろそれを前提としている）
- この課題では，`np.linalg.qr()`や`scipy.linalg.qr()`は**使用せず**，資料で説明された**ハウスホルダー変換**と**ギブンス変換**を用いること
- ただし，レイリー商反復`rayleigh_quotient_iteration()`を使用する際には，非正則行列によるエラーを防ぐ必要がある．`try/catch`を使用して例外を捕捉し，反復を適切に停止させることが必要かもしれない．

**実行手順**

1. $n$を2から10程度の範囲でランダムに変更し，いくつかのランダム行列を生成する．
2. 実装した`eigh_via_qr_rayleigh()`を使用して得られるQR分解が，`np.linalg.qr`（もしくは`scipy.linalg.qr`）と同じ結果であるか確認する．一致しない場合は，その原因を検討する．
   1. 固有値の一致を確認する際には，固有値の並び順（昇順または降順）を揃えること
   2. 固有ベクトルの一致を確認する際には，符号を除いて一致するかを確認すること


## 1.2. <a id='toc1_2_'></a>[課題05-2：ハウスホルダー変換によるQR分解](#toc0_)

この課題ではハウスホルダー変換によるQR分解を実装する．


### 1.2.1. <a id='toc1_2_1_'></a>[2つのハウスホルダー変換](#toc0_)

授業資料ではハウスホルダー変換を紹介したが，
この変換は，異なる目的に対して適用することができる．
一つは授業資料で説明した「対称行列を三重対角行列に変換する」ためのハウスホルダー変換であり，
もう一つがこの課題で実装する「一般の行列をQR分解する」ためのハウスホルダー変換である．


#### 1.2.1.1. <a id='toc1_2_1_1_'></a>[「対称行列を三重対角行列に変換する」ためのハウスホルダー法の復習](#toc0_)

授業資料で紹介した
「対称行列を三重対角行列に変換する」ハウスホルダー法では，
以下のように，鏡映変換$Q_1$
\begin{align*}
Q_1 &=
\begin{pmatrix}
1 &  \\
 & H_1 \\
\end{pmatrix}
\in R^{n \times n}
\end{align*}
を左右から対称行列$A$（つまり$a_{ij} = a_{ji}$）にかけて，
$A$の第1列の第3要素以下（との第1行の第3要素から右）を0にすることを考えた．

\begin{align*}
Q_1 A Q_1 =
Q_1
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
a_{31} & a_{32} & \cdots & a_{3n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix}
Q_1
=
\begin{pmatrix}
a_{11} & * & 0 & \cdots & 0 \\
* & * & * & \cdots & * \\
0 & * & * & \cdots & * \\
\vdots & \vdots & & \ddots & \vdots \\
0 & * & * & \cdots & *
\end{pmatrix}
\end{align*}

同様にして，次は鏡映変換$Q_2$
\begin{align*}
Q_2 &=
\begin{pmatrix}
1 &  \\
  & 1 \\
  &   & H_2 \\
\end{pmatrix}
\in R^{n \times n}
\end{align*}
を左右からかけて$Q_1 A Q_1$の第2列の第4要素以下（と第2行の第4要素より右）を0にし，
その次は鏡映変換$Q_3$を左右からかけて$Q_2 Q_1 A Q_1 Q_2$の第3列の第5要素以下（と第3行の第5要素より右）を0にし（以下同様），
という処理を繰り返した．


三重対角行列に変換するためにはこのような$Q_{\mathrm{hh}} = Q_1 Q_2 \cdots Q_{n-2}$を左右からかけて

$$
A_{\mathrm{tri}} = Q_{\mathrm{hh}}^T A Q_{\mathrm{hh}}
$$

と$A$を三重対角行列へ変換した．



#### 1.2.1.2. <a id='toc1_2_1_2_'></a>[「一般の行列をQR分解する」ためのハウスホルダー変換](#toc0_)

これと同じ要領で「一般の行列をQR分解する」ためにハウスホルダー変換を行うには，
以下のように，鏡映変換$Q_1$
\begin{align*}
Q_1 &= H_1
\in R^{n \times n}
\end{align*}
を左から一般の行列$A$（つまり$a_{ij} \neq a_{ji}$）にかけて，$A$の第1列の第2要素以下を0にすることを考える．

\begin{align*}
Q_1 A =
Q_1
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
a_{31} & a_{32} & \cdots & a_{3n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1} & a_{n2} & \cdots & a_{nn}
\end{pmatrix}
=
\begin{pmatrix}
* & * & \cdots & * \\
0 & * & \cdots & * \\
0 & * & \cdots & * \\
\vdots & \vdots & \ddots & \vdots \\
0 & * & \cdots & *
\end{pmatrix}
\end{align*}

同様にして，次は鏡映変換$Q_2$
\begin{align*}
Q_2 &=
\begin{pmatrix}
1 &  \\
 & H_2 \\
\end{pmatrix}
\in R^{n \times n}
\end{align*}
を左から$A$にかけて$Q_1 A$の第2列の第3要素以下を0にし，
その次は鏡映変換$Q_3$を左からかけて$Q_2 Q_1 A$の第3列の第4要素以下を0にし，
という処理を繰り返す．

上三角行列$R$に変換するためにはこのような$Q_1, Q_2, \cdots, Q_{n-1}$を$A$の左からかけて
\begin{align*}
Q_{n-1} \cdots Q_2 Q_1 A &= R
\end{align*}
する．$A$をQR分解するには
\begin{align*}
A &= (Q_{n-1} \cdots Q_2 Q_1)^{-1} R \\
&= (Q_1 Q_2 \cdots Q_{n-1}) R \\
&= QR
\end{align*}
とする．


### 1.2.2. <a id='toc1_2_2_'></a>[課題内容](#toc0_)

行列 $A \in \mathbb{R}^{n \times n}$ を，ハウスホルダー変換を用いて QR 分解する関数 `qr_via_housholder()` を作成する．

**やること**

1. ハウスホルダー変換を繰り返して上三角化する方法を用いる．
   - 上記で説明したように，ハウスホルダー変換を繰り返して上三角化する方法を用いること．
   - 各ステップでは，ハウスホルダー変換 $Q_k$ を左から掛けて，列の下側の成分を0にしていく．
2. 関数は直交行列 $Q$ と上三角行列 $R$ を返す．
   - QR 分解は符号が一意ではないため，最終的に返す上三角行列 $R$ の対角成分が正になるように，対応する列の符号調整を行うこと．
   - ハウスホルダー変換のコードは，対称行列を三重対角化する場合の授業資料実装を参考にすること．

**関数定義**

```python
def qr_via_housholder(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray]:
```

**実行手順**

1. $ n $ を2から10程度にランダムに変更し，いくつかのランダムな行列 $ A $ を生成する．
2. 実装した `qr_via_housholder(A)` の戻り値$Q, R$と，`np.linalg.qr(A)` または `scipy.linalg.qr(A)` の戻り値を比較し，一致することを確認する．
   - 一致していない場合には，符号の取り扱いや正規化に問題がある可能性があるため，原因を考察する．

**注意点**

- `np.linalg.qr` や `scipy.linalg.qr` を使用しないこと．


## 1.3. <a id='toc1_3_'></a>[課題05-3：QR分解を用いたthin SVD](#toc0_)

授業資料では，固有値分解（対角化）を用いたthin SVDの単純な計算方法を示した．
これ以外にも，QR分解を用いた単純な計算方法がある．その前にQR分解の種類について説明する．


### 1.3.1. <a id='toc1_3_1_'></a>[full QR分解とreduced QR分解](#toc0_)

授業資料での$A$のQR分解の説明は，$A$が正方行列である場合を想定していた．
しかし$A$が非正方行列の場合にもQR分解は可能であり，サイズによって以下のように分類される．


#### 1.3.1.1. <a id='toc1_3_1_1_'></a>[QR分解：$m=n$の場合](#toc0_)

正方行列$A \in R^{n \times n}$に対するQR分解は
$$
A = QR
$$
である．ここで$Q \in R^{n \times n}$は直交行列，$R \in R^{n \times n}$は上三角行列である．

この実装は上記のハウスホルダー変換によるQR分解で可能である．


#### 1.3.1.2. <a id='toc1_3_1_2_'></a>[QR分解：$m>n$の場合](#toc0_)

$m>n$である一般の縦長の行列$A \in R^{m \times n}$に対するQR分解も同様に
$$
A = QR
$$
であり，ここで$Q \in R^{m \times m}$は直交行列である．
$R \in R^{m \times n}$は，上三角行列部分$R_1 \in R^{n \times n}$と零行列部分$\mathrm{O} \in R^{(m-n) \times n}$からなる行列
\begin{align*}
R =
\begin{pmatrix}
R_1 \\ \mathrm{O}
\end{pmatrix}
\end{align*}
であり，
これは上台形行列（upper trapezoidal matrix）と呼ばれる行列であり，
主対角（ここでは$R$は非正方なので$R_1$の主対角を指す）よりも下の成分が0の行列である
（QR分解の文脈では上三角行列と呼ばれていることもある）．

ここで$Q$を，$R$の$R_1$に対応する列$Q_1  \in R^{m \times n}$と，
零行列部分に対応する列$Q_2 \in R^{m \times (m-n)}$に以下のように分解する．
\begin{align*}
Q = (Q_1, Q_2)
\end{align*}
すると
\begin{align*}
A = QR = Q_1 R_1
\end{align*}
となり，小さい行列の積で表せることになる．
元々の$A = QR$はfull QR分解やcomplete QR分解，
$A = Q_1 R_1$はreduced QR分解やeconomic QR分解などと呼ばれている．

実装は$m=n$の場合のハウスホルダー変換とほぼ同じであり，
$Q_1, Q_2, \cdots, Q_{n-1}, Q_n$（$n$まで）を左からかけて要素を0にする．
reduced QRの場合には`Q`の最初の$n$列，`R`の最初の$n$行を返す．

#### 1.3.1.3. <a id='toc1_3_1_3_'></a>[QR分解：$m<n$の場合](#toc0_)

$m<n$である一般の横長の行列$A \in R^{m \times n}$に対するQR分解も同様に
$$
A = QR
$$
であり，ここで$Q \in R^{m \times m}$は直交行列である．
$R \in R^{m \times n}$は，上三角行列部分$R_1$と正方行列$R_2$からなる
\begin{align*}
R = (R_1, R_2)
\end{align*}
であり，これも上台形行列である
（ここでは$R$は横長の非正方であるが$R_1$の主対角よりも下は0であり，$R_2$には依存しない）．

この場合には零行列部分はないため，full/completeとreduced/economicの違いはない．

実装は$m=n$の場合のハウスホルダー変換と同じであり，
$Q_1, Q_2, \cdots, Q_{m-1}$を左からかけて要素を0にする．


#### 1.3.1.4. <a id='toc1_3_1_4_'></a>[reduced QR分解のまとめ](#toc0_)

一般の行列$A \in R^{m \times n}$に対するreduced QR分解
$$
A = QR
$$
は，$p = \min(m, n)$として

- $Q \in R^{m \times p}$は直交行列（$m \le n$の場合）または列直交行列（$m > n$の場合）
- $R \in R^{p \times n}$は上三角行列（$m = n$の場合）または上台形行列（$m \neq n$の場合）

である．


#### 1.3.1.5. <a id='toc1_3_1_5_'></a>[numpy/scipyの実装](#toc0_)

numpyとscipyのQR分解の関数には，full/complete QRとreduced/economic QRのどちらの分解の結果を返すのかのフラグ（`mode`）を引数に取る．
なお$m<n$の場合には，どちらの`mode`でも返り値は同じである．
なお現在のバージョンでは，numpyとscipyで`mode`の**デフォルトが異なっている**ので注意が必要である．

- [`numpy.linalg.qr()`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.qr.html)
  > Compute the qr factorization of a matrix.
  - `mode="complete"`: complete QR分解（$Q, R$）の結果を返す
  - `mode="reduced"`: reduced/economic QR分解（$Q_1, R_1$）の結果を返す（デフォルト）
- [`scipy.linalg.qr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.qr.html)
  > Compute QR decomposition of a matrix.
  - `mode="full"`: full QR分解（$Q, R$）の結果を返す（デフォルト）
  - `mode="economic"`: economic/reduced QR分解（$Q_1, R_1$）の結果を返す


上記の関数の内部的には，LAPACKの関数を利用している．

- `numpy.linalg.qr()`
  - [dgeqrf](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dgeqrf.f)
    <!-- > DGEQRF computes a QR factorization of a real M-by-N matrix A -->
- `scipy.linalg.qr()`
  - [dgeqrf](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dgeqrf.f)
    <!-- > DGEQRF computes a QR factorization of a real M-by-N matrix A: -->
  - [dgeqp3](https://github.com/Reference-LAPACK/lapack/blob/master/SRC/dgeqp3.f)
    <!-- > DGEQP3 computes a QR factorization with column pivoting of a
    > matrix A:  A*P = Q*R  using Level 3 BLAS.
    - ピボット付きQR分解
      - [Gene H. Golub, Charles F. Van Loan, Matrix Computations, 4th edition, Johns Hopkins Univ Press, 2012](https://www.press.jhu.edu/books/title/10678/matrix-computations), p.276, 5.4.2, QR with Column Pivoting. -->

- [LAPACK Users' Guide Third Edition, QR Factorization](https://www.netlib.org/lapack/lug/node69.html)
  > The traditional algorithm for QR factorization is based on the use of elementary Householder matrices


### 1.3.2. <a id='toc1_3_2_'></a>[full/reduced QR分解を利用したthin SVDの単純な計算方法](#toc0_)


行列$A \in R^{m \times n}$の特異値分解$A=U \Sigma V^T$を計算するための単純な方法の一つに，
full/reduced QR分解を用いる以下の単純な方法がある．

- $m > n$の場合：
  1. $A^T A \in R^{n \times n}$の対角化を計算し，$V^T (A^T A) V = \Sigma^2$を得る．
     - これで$V$と$\Sigma$が得られる．
     - ここで$V \in R^{n \times n}$は直交行列，$\Sigma \in R^{n \times n}$は対角行列である．
  2. $A V \in R^{m \times n}$をQR分解して$AV = U\Sigma$を得る．
     - これで$U$が得られる．
     - ここで$U \in R^{m \times n}$は列直交行列（これが$Q$），$\Sigma \in R^{n \times n}$は対角行列（これが$R$）である．
  3. SVD結果$A = U \Sigma V^T$を出力する．
- $m < n$の場合：
  1. $A A^T \in R^{m \times m}$の対角化を計算し，$U^T (A A^T) U = \Sigma^2$を得る．
     - これで$U$と$\Sigma$が得られる．
     - ここで$U \in R^{m \times m}$は直交行列，$\Sigma \in R^{m \times m}$は対角行列である．
  2. $A^T U \in R^{n \times m}$をreduced/economic QR分解して$A^T U = V\Sigma$を得る．
     - これで$V$が得られる．
     - ここで$V \in R^{n \times m}$は列直交行列（これが$Q$），$\Sigma \in R^{m \times m}$は対角行列（これが$R$）である．
  3. SVD結果$A = U \Sigma V^T$を出力する．


これは固有値分解（のみ）を利用したthin SVDの単純な計算方法に比べて，QR分解で求めた$m > n$の場合の$U$（または$m < n$の場合の$V$）の直交性がよい．

- [Gene H. Golub, Charles F. Van Loan, Matrix Computations, 4th edition, Johns Hopkins Univ Press, 2012](https://www.press.jhu.edu/books/title/10678/matrix-computations), 8.6.3 The SVD Algorithm, p.488.


### 1.3.3. <a id='toc1_3_3_'></a>[課題内容](#toc0_)

行列 $A \in \mathbb{R}^{m \times n}$ に対して，以下の2つの方法で thin SVD を計算する関数を実装する．

- QR分解を利用する関数`svd_via_qr()`：上記の「full/reduced QR分解を利用したthin SVDの単純な計算方法」
- 固有値分解を利用する関数`svd_via_eigh()`：授業資料の「固有値分解を利用した単純なthin SVDの計算手法」で説明した，固有値分解（のみ）を用いるthin SVDの単純な計算方法

**関数定義**

```python
def svd_via_qr(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
```

```python
def svd_via_eigh(
    A: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
```

いずれの関数も，引数と返り値は以下の通り．

- 実装する関数は引数に行列$A$をとる．
- 実装する関数の返り値は，$A$の特異値分解$U \Sigma V^T$の結果である$U, \Sigma, V^T$である．
    - $\Sigma$は対角行列ではなく，対角要素のベクトルをndarrayとして返す．
    - $V^T$は，$V$ではなく$V^T$を返す．

**注意事項**

この課題では，既存ライブラリの使い方を習得することも目指す．したがって，

- 授業資料のコードや課題で作成したQR分解や固有値計算の関数は**使用しないこと**．
- full/reduced QR分解には`np.linalg.qr()`または`scipy.linalg.qr()`を**使用すること**．
- 固有値分解には`np.linalg.eigh()`または`scipy.linalg.eigh()`を**使用すること**．
  - 対称行列に対しては，`eig()`は使用せず，`eigh()`を用いること



**実行手順**

- $m, n$を2から10程度にランダムに変更し，いくつかのランダムな行列を生成する
- 実装した2つの関数が`np.linalg.svd()`（または`scipy.linalg.svd()`）と同じ結果を返すことを確かめる．
   - 特異値の一致を確認する際には，特異値の並び順（昇順または降順）を揃えること
   - 特異ベクトルの一致を確認する際には，符号を除いて一致するかを確認すること

**実装上の注意点**

- 固有値分解や特異値分解における`U`や`V`の転置は，バグの原因となりやすい．また，特異値と固有値の順序（昇順・降順）が逆転していることにも注意が必要である．


## 1.4. <a id='toc1_4_'></a>[課題05-4：ゲルシュゴリンの定理による固有値の存在範囲の可視化](#toc0_)

**ゲルシュゴリンの定理**によれば，複素正方行列$A \in \mathbb{C}^{n \times n}$の複素固有値は，次のような**ゲルシュゴリン円盤**（Gershgorin disk）の和集合$\cup_i D_i \subset \mathbb{C}$に存在する．

- 各円盤$i$の中心は行列の対角成分$a_{ii} \in \mathbb{C}$
- 半径は対応する行$i$にある非対角成分の絶対値和$R_i = \sum_{j \neq i} |a_{ij}|$

なお実対称行列の場合は，固有値は実数であるため，ゲルシュゴリンの円盤は実軸上の区間として考えることができる．

- https://ja.wikipedia.org/wiki/%E3%82%B2%E3%83%AB%E3%82%B7%E3%83%A5%E3%82%B4%E3%83%AA%E3%83%B3%E3%81%AE%E5%AE%9A%E7%90%86
    > 数学におけるゲルシュゴリンの定理は正方行列の固有値の大まかな存在範囲を示す。

この課題では，ゲルシュゴリンの円盤と実際の固有値を可視化し，
ゲルシュゴリンの円盤がどの程度正確に固有値の存在範囲を表しているかを考察する．


**やること**

1. 実対称行列$A \in \mathbb{R}^{n \times n}$の実軸上のゲルシュゴリン円盤（区間）を計算し，可視化する．
   - 以下の手順を関数`gershgorin_intervals()`として実装する．
     - 各区間の中心$a_{ii}$に対して半径$R_i$（区間幅の半分）を求める．
     - $a_{ii} - R_i$から$a_{ii} + R_i$までの区間を求める．
     - 区間のリストを返す
   - $A$の固有値を`np.linalg.eigvalsh()`で求める．
   - 区間と固有値をプロットし，固有値がすべてゲルシュゴリン円盤（区間）内に存在することを確認する．

2. 非対称行列$B \in \mathbb{C}^{m \times n}$のゲルシュゴリン円盤を計算し，可視化する．
   - 以下の手順を関数`gershgorin_disks()`として実装する．
     - 各円盤の中心$a_{ii}$に対して半径$R_i$を求める．
     - 円盤の中心（複素数）と半径（実数）を返す
   - $B$の固有値を`np.linalg.eigvals()`で求める．
   - 区間と固有値をプロットし，固有値がすべてゲルシュゴリン円盤内に存在することを確認する．
     - 2次元平面を複素平面とみなして2Dプロットを作成する

**関数定義**

```python
def gershgorin_intervals(
    A: np.ndarray
) -> List[Tuple[float,float]]:

def gershgorin_disks(
    B: np.ndarray
) -> List[Tuple[complex,float]]:
```


**実行手順**

1. $n=3\sim10$ 程度のランダムな実対称行列 $A$ を生成する．
   1. ゲルシュゴリン区間と実際の固有値を計算し，両者をプロットする．
2. $n=3\sim10$ 程度のランダムな非対称行列 $B$ を生成する．
   1. ゲルシュゴリン円盤と実際の固有値を計算し，複素平面に可視化する．
3. 得られた可視化結果を比較し，ゲルシュゴリンの定理が固有値の存在範囲を正しく与えていることを確認する．
4. ゲルシュゴリンの定理で得られた固有値の存在範囲が，固有値計算に役立てることができるのかどうかを考察・議論する．

**可視化の例**

可視化の方法は様々なものが考えられるため，分かりやすい可視化方法を工夫すること．
（以下の例は一例であり，これに限らない）

以下は実数区間の可視化の例である．存在区間を`ax.axvspan`で，固有値を`ax.vline`でプロットしている．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6, 1))
ax = fig.subplots()

# 数値例
centers = np.array([1.0, 3.5, -0.2, 5.3])
radii = np.array([1.6, 0.8, 2.2, 0.6])
eigs = np.array([0.7, 3.5, -0.4, 5.55])

# 区間の可視化
for c, r in zip(centers, radii):
    ax.axvspan(c - r, c + r, alpha=0.12, color="red")

# 固有値の可視化
ax.vlines(eigs, 0, 1, colors="black", linewidth=1.2)

ax.set_yticks([])
plt.tight_layout()
plt.show()

以下は複素円盤の可視化の例である．円盤を`Circle`と`ax.add_patch`で，固有値を`scatter`でプロットしている．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

fig = plt.figure(figsize=(6, 6))
ax = fig.subplots()

# 数値例
centers = np.array([1.0 + 0.5j, 3.5 - 0.2j, -0.2 + 1.3j, 5.3 + 0.0j], dtype=complex)
radii = np.array([1.6, 0.8, 2.2, 0.6])
eigs = np.array([1.9 + 0.1j, 3.5 - 0.4j, -1.4 + 1.0j, 5.55 - 0.2j], dtype=complex)

# 円盤の可視化
for c, r in zip(centers, radii):
    circ = Circle(
        (c.real, c.imag), r,
        facecolor='red', alpha=0.12, edgecolor='k')
    ax.add_patch(circ)

# 固有値の可視化
ax.scatter(eigs.real, eigs.imag, color='k', label="eigenvalues")

ax.set_xlabel("real axis")
ax.set_ylabel("imag axis")
ax.set_aspect('equal')
ax.legend()

plt.tight_layout()
plt.show()
